# BENCHMARKING



Carga de Variables de entorno

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import sys
import os

# Obtener el directorio raíz (el que contiene tanto "src" como "benchmarking")
root_dir = os.path.abspath(os.path.join(os.path.dirname("Pipeline_Benchmarks.ipynb"), '..'))

# Agregar el directorio raíz al PYTHONPATH
sys.path.append(root_dir)

### Pipeline


In [ ]:
from src.loaders.load import load_pdf
from src.chunking.chunk import generate_chunks
from src.chunking.semantic_chunk import generate_semantic_chunk
from src.chunking.semantic_chunk2 import generate_semantic_chunks
from src.embedding.embedding import generate_embeddings
#from src.vector_store_client.qdrant import create_qdrant_vector_store
from src.vector_store_client.qdrant_hybrid import create_qdrant_vector_store_hybrid
from src.vector_store_client.hybrid import run_hybrid_search
from src.retrievers.retrieve import retrieve_documents
from src.retrievers.rank import rank_results
from src.augmented.llm import generate_augmented_response

# Cargar documentos
file_path = "../data/biblioteca-de-alimentos2.pdf"
#"Castro_Cofre_Zurita/data/biblioteca-de-alimentos.pdf"
documents = load_pdf(file_path)

# Dividir en chunks
output_folder = "..data/chunks/"
#chunks = generate_chunks(documents)
chunks = generate_semantic_chunk(documents, chunk_size=215,chunk_overlap=0)
#chunks = generate_semantic_chunks(documents, percentile=79)
# Generar embeddings y obtener el modelo de embeddings

embeddings, embedding_model = generate_embeddings(chunks)

# Crear base vectorial usando el modelo de embeddings

#vector_store = create_qdrant_vector_store(chunks, embedding_model)
#vector_store = create_qdrant_vector_store(chunks, embedding_model)

# Consultar la base vectorial
query = "Quais são as normas para lactantes no Brasil?"
#"O que fala acerca do rotulagem?"
#"Quais são as normas para lactantes no Brasil?"

# devuelve resultados con hybrid search
#results = retrieve_documents(query, vector_store,2)
results, qdrant = run_hybrid_search(chunks, embedding_model, query)

#print("Results passed to Reranker:", [doc.page_content for doc in results])


Fetching 29 files: 100%|██████████| 29/29 [00:00<?, ?it/s]


Results passed to Reranker: ['11.265, de 3 de janeiro de 2006, que regulamenta a comercialização de alimentos para lactentes e \ncrianças de primeira infância e a de produtos de puericultura correlatos, e dá outras providências.', 'e Boas Práticas  \n3.1. Boas Práticas de Fabricação (BPF) para estabelecimentos industrializadores de \nalimentos \n \nTema Regulatório 3.26 da Agenda Regulatória 202 4/2025: Revisão das normas de Boas práticas de alimentos \n \nTema Regulatório 3.26 da Agenda Regulatória 202 4/2025: Revisão das normas de Boas práticas de  \nfabricação (BPF) para estabelecimentos industrializadores de alimentos.', 'Decreto-Lei 986/1969 – Institui normas básicas sobre alimentos. \nLei 11.265/2006 – Regulamenta a comercialização de alimentos para lactentes e crianças de primeira infância Lei 11.265/2006 – Regulamenta a comercialização de alimentos para lactentes e crianças de primeira infância \ne também a de produtos de puericultura correlatos. \n Alterada por: \nLei 11.474/2

Retriever

In [7]:
retriever = qdrant.as_retriever()

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import OllamaLLM
from langchain_openai import ChatOpenAI


#llm = ChatOpenAI(model="gpt-4-turbo-preview") 

llm = ChatOpenAI(model="gpt-3.5-turbo") 

# Define prompt template
template = """Utilize the retrieved context below to answer the question.
If you're unsure of the answer, simply state you don't know and apologies in portuguese
Keep your response concise, limited to two sentences.
Question: {question}
Context: {context}
"""

prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [9]:
rag_chain.invoke("What is the purpose of the regulation?")

'The purpose of the regulation is to establish standards and quality criteria for food products and to oversee registration, inspection, and risk control activities in the food sector.'

### 2.- Pasos previos a la Evaluación

a) Creación del Dataset de preuntas y respuestas

In [10]:
QA_generation_prompt = ChatPromptTemplate.from_template("""
Your task is to write a factoid question in and an answer given a context in portuguese.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}
Output:::""")

# Create a chain to create a question
question_chain = (
    {"context": RunnablePassthrough()}
    | QA_generation_prompt
    | llm
    | StrOutputParser()
)

b) Obtener una Muestra Aleatoria de documentos

In [11]:
import random
from tqdm import tqdm

# Sample 15 documents to generate questions
sampled_docs = random.sample(documents, 15)

# Generate questions for each document
sampled_docs_processed = [doc.page_content for doc in sampled_docs]

c) Generar Preguntas y respuestas

In [12]:
# Generate questions

questions = [question_chain.invoke({"context": sampled_context}) for sampled_context in tqdm(sampled_docs_processed)]

100%|██████████| 15/15 [00:16<00:00,  1.07s/it]


Visualización de preguntas y respuestas

In [13]:
questions

['Factoid question: Qual é o número do guia que trata da doação de alimentos com segurança sanitária?\nAnswer: Guia nº 57, versão 1, de 07/11/2022 - Guia para Doação de Alimentos com Segurança Sanitária.',
 'Factoid question: Qual é o tema regulatório da Agenda Regulatória 2024/2025 relacionado à revisão da regulamentação de películas de celulose regeneradas em contato com alimentos?\nAnswer: Tema Regulatório 3.20 da Agenda Regulatória 2024/2025: Revisão da regulamentação de películas de celulose regeneradas em contato com alimentos.',
 'Factoid question: Qual é o tema regulatório 3.5 da Agenda Regulatória 2024/2025?\nAnswer: Reavaliação da autorização de uso do aditivo alimentar dióxido de titânio em alimentos.',
 'Factoid question: Quais são os requisitos sanitários para alimentos para fins especiais de acordo com o tema regulatório 3.27 da Agenda Regulatória 2024/2025?\nAnswer: Requisitos sanitários para alimentos para fins especiais de acordo com o tema regulatório 3.27 da Agenda R

Parsing de la Preguntas

In [14]:
questions_processed = []
ground_truth = []
for question in questions:
    questions_processed.append(question.split("Factoid question: ")[-1].split("Answer: ")[0])
    ground_truth.append(question.split("Factoid question: ")[-1].split("Answer: ")[1])

In [15]:
contexts = []
answers = []
# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])


C:\Users\nicol\AppData\Local\Temp\ipykernel_29328\1022487351.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])


d) Generación de Diccionario

In [16]:
data = {
    "question": questions,
    "answer": answers,
    "reference": ground_truth,
    "retrieved_contexts": contexts
}

In [53]:
#!pip install datasets

In [17]:
from datasets import Dataset

# Convert dict to dataset
dataset = Dataset.from_dict(data)

### 3. Evaluación de Métricas on RAGAS

In [55]:
# !pip intall ragas

In [18]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness, # Measures how well the model generates answers that are faithful to the context
    answer_relevancy, # Measures how well the model generates answers that are relevant to the question
    context_recall, # Context recall measures how well the model retrieves relevant context
    context_precision, # Context precision measures how well the model retrieves only relevant context
)

In [19]:
result = evaluate(
    dataset = dataset,
    llm=llm,
    embeddings=embedding_model,
    metrics=[
        context_recall,
        faithfulness,
        answer_relevancy,
        context_precision,
    ],)

df = result.to_pandas()
df

Evaluating: 100%|██████████| 60/60 [00:19<00:00,  3.15it/s]


,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,answer_relevancy,context_precision
0,Factoid question: Qual é o número do guia que ...,"[para o consumo humano. \nGuia nº 26, versão 2...","Guia nº 57, versão 1, de 07/11/2022 - Guia par...","Guia nº 57, versão 1, de 07/11/2022 - Guia par...",1.00,0.25,0.604720,0.750000
1,Factoid question: Qual é o tema regulatório da...,[metálicos em contato com alimentos. \n \nTe...,Tema Regulatório 3.20 da Agenda Regulatória 20...,Tema Regulatório 3.20 da Agenda Regulatória 20...,1.00,1.00,0.426825,1.000000
2,Factoid question: Qual é o tema regulatório 3....,[de tecnologia \n \nTema Regulatório 3.5 da A...,Reavaliação da autorização de uso do aditivo a...,Reavaliação da autorização de uso do aditivo a...,1.00,1.00,0.329241,0.833333
3,Factoid question: Quais são os requisitos sani...,[Tema Regulatório 3.27 da Agenda Regulatória 2...,Os requisitos sanitários para alimentos para f...,Requisitos sanitários para alimentos para fins...,1.00,1.00,0.706628,1.000000
4,Factoid question: Quais normas foram alteradas...,[para uso em alimentos. Altera a RDC 22/2015. ...,RDC 22/2015.,RDC 22/2015.,1.00,1.00,0.268661,0.916667
5,Factoid question: Quais são os materiais autor...,[RDC 52/2010 – Corantes em embalagens e equipa...,"Lista positiva de monômeros, outras substância...","Lista positiva de monômeros, outras substância...",1.00,1.00,0.915515,1.000000
6,Factoid question: Qual é o tema regulatório 3....,[1.12. Requisitos sanitários para suplementos ...,Atualização periódica da lista de constituinte...,Atualização periódica da lista de constituinte...,1.00,0.50,0.244082,0.833333
7,Factoid question: Qual é o tema regulatório 3....,[Tema Regulatório 3.8 da Agenda Regulatória 20...,Regulamentação das listas de novos alimentos e...,Regulamentação das listas de novos alimentos e...,1.00,1.00,0.462661,1.000000
8,Factoid question: Quais são os temas regulatór...,[Tema Regulatório 3.9 da Agenda Regulatória 20...,"Tema Regulatório 3.3, 3.6, 3.9 e 3.16 da Agend...","Tema Regulatório 3.3, 3.6, 3.9 e 3.16 da Agend...",0.75,0.50,0.000000,1.000000
9,Factoid question: Qual legislação obriga que o...,[de 2003. \n \nLei 10.674/2003 – Obriga a que...,Lei 10.674/2003.,Lei 10.674/2003,1.00,1.00,0.000000,0.833333


Generación de Archivo CSV con resultados

In [20]:
df.to_csv("results/pipeline_ragas_results.csv", index=False)

Promedio de Métricas

In [21]:
# get mean of the metrics column by column
print("Mean Faithfulness: ", round(df["faithfulness"].mean(), 4))
print("Mean Answer relevancy: ", round(df["answer_relevancy"].mean(), 4))
print("Mean Context recall: ", round(df["context_recall"].mean(), 4))
print("Mean Context precision: ", round(df["context_precision"].mean(), 4))


Mean Faithfulness:  0.7433
Mean Answer relevancy:  0.3302
Mean Context recall:  0.9833
Mean Context precision:  0.9333


### 4. Agregando un paso de 'Rerankeo'

Esto permite reordenar los documentos de acuerdo a su relevancia semantica

In [22]:
query = "What is the purpose of the regulation?"

retrieved_docs = retriever.get_relevant_documents(query, kwargs={"k": 10})

In [23]:
retrieved_docs

[Document(metadata={'_id': 'b0c7a990bbbd48d29bfde40a85312331', '_collection_name': 'hybrid_search_collection'}, page_content='No setor de alimentos, a Anvisa coordena, supervisiona e controla as atividades de registro, inspeção, \nfiscalização e controle de riscos, sendo responsável por estabelecer normas e padrões de qualidade e fiscalização e controle de riscos, sendo responsável por estabelecer normas e padrões de qualidade e \nidentidade a serem observados.'),
 Document(metadata={'_id': 'a0982d91318e41da8d0d7b6332e4eb77', '_collection_name': 'hybrid_search_collection'}, page_content='(2019-nCoV). \n1.1. Procedimentos para regularização de alimentos \n \nTema Regulatório 3.22 da Agenda Regulatória 2024/2025: Revisão da regulamentação sobre regularização  \nde alimentos dispensados de registro.'),
 Document(metadata={'_id': 'e2aad1a8da8343d9a40caaf65767da4d', '_collection_name': 'hybrid_search_collection'}, page_content='1.12. Requisitos sanitários para suplementos alimentares \n \nT

Mediante Cohere (no utilizado)

In [62]:
# import cohere as co
# cohere_client = co.Client(os.getenv("COHERE_API_KEY"))
# def rerank_docs(query, retrieved_docs):
#     reranked_docs = cohere_client.rerank(
#         model="rerank-english-v3.0",
#         query=query,
#         documents=retrieved_docs,
#         rank_fields=["page_content"],
#         return_documents=True
#     )
#     return reranked_docs


Mediante Rerankers

In [26]:
from rerankers import Reranker

def open_source_reranker(query, retrieved_docs):
    reranker = Reranker('cross-encoder', verbose=0,model_type='cross-encoder',lang='pt')
    #reranker = Reranker("colbert") # colber model used for reranking
    retrieved_docs = [doc.page_content for doc in retrieved_docs]
    reranked_docs = reranker.rank(query, retrieved_docs)
    return reranked_docs


In [27]:
reranked_docs = open_source_reranker(query, retrieved_docs)

Loading default cross-encoder model for language pt


In [28]:
reranked_docs.results

[Result(document=Document(document_type='text', text='No setor de alimentos, a Anvisa coordena, supervisiona e controla as atividades de registro, inspeção, \nfiscalização e controle de riscos, sendo responsável por estabelecer normas e padrões de qualidade e fiscalização e controle de riscos, sendo responsável por estabelecer normas e padrões de qualidade e \nidentidade a serem observados.', base64=None, image_path=None, doc_id=0, metadata={}), score=-4.045792102813721, rank=1),
 Result(document=Document(document_type='text', text='(2019-nCoV). \n1.1. Procedimentos para regularização de alimentos \n \nTema Regulatório 3.22 da Agenda Regulatória 2024/2025: Revisão da regulamentação sobre regularização  \nde alimentos dispensados de registro.', base64=None, image_path=None, doc_id=1, metadata={}), score=-6.328507900238037, rank=2),
 Result(document=Document(document_type='text', text='1.12. Requisitos sanitários para suplementos alimentares \n \nTema Regulatório 3.29 da Agenda Regulatór

In [29]:
contexts = []
answers = []
# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    retrieved_docs = retriever.get_relevant_documents(query)
    reranked_docs = open_source_reranker(query, retrieved_docs)
    if reranked_docs.results:  # Check if there are any results
        contexts.append([reranked_docs.results[0].document.text])

data = {
    "question": questions,
    "answer": answers,
    "reference": ground_truth,
    "retrieved_contexts": contexts
}

Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt
Loading default cross-encoder model for language pt


In [33]:
reranked_dataset = Dataset.from_dict(data)
result = evaluate(
    dataset = reranked_dataset,
    llm=llm,
    embeddings=embeddings,
    metrics=[
        context_recall,
        faithfulness,
        answer_relevancy,
        context_precision,
    ],)
reranked_df = result.to_pandas()
reranked_df.to_csv("results/reranked_pipeline_ragas_results.csv", index=False)


Evaluating: 100%|██████████| 60/60 [00:20<00:00,  2.90it/s]


In [32]:
# get mean of the metrics column by column
print("Mean Faithfulness: ", round(reranked_df["faithfulness"].mean(), 4))
print("Mean Answer relevancy: ", round(reranked_df["answer_relevancy"].mean(), 4))
print("Mean Context recall: ", round(reranked_df["context_recall"].mean(), 4))
print("Mean Context precision: ", round(reranked_df["context_precision"].mean(), 4))

Mean Faithfulness:  0.6429
Mean Answer relevancy:  0.3811
Mean Context recall:  1.0
Mean Context precision:  0.8667
